# Nova seção

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalando o PySpark
!apt-get -qq update
!apt-get -qq install -y openjdk-17-jdk-headless
!pip -q install -U pyspark[connect]==4.0.0

print('✅ PySpark instalado com sucesso!')


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../openjdk-17-jdk-headless_17.0.18+8-1~22.04.1_amd64.deb ...
Unpacking openjdk-17-jdk-headless:amd64 (17.0.18+8-1~22.04.1) over (17.0.17+10-1~22.04) ...
Preparing to unpack .../openjdk-17-jre-headless_17.0.18+8-1~22.04.1_amd64.deb ...
Unpacking openjdk-17-jre-headless:amd64 (17.0.18+8-1~22.04.1) over (17.0.17+10-1~22.04) ...
Setting up openjdk-17-jre-headless:amd64 (17.0.18+8-1~22.04.1) ...
Installing new version of config file /etc/java-17-openjdk/jfr/default.jfc ...
Installing new version of config file /etc/java-17-openjdk/security/java.security ...
Setting up openjdk-17-jdk-headless:amd64 (17.0.18+8-1~22.04.1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 956.8 kB/s eta 0:00:00
  Installi

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Criando uma SparkSession
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('PrevisaoVendasML') \
    .config('spark.driver.memory', '4g') \
    .getOrCreate()

print('✅ SparkSession criada com sucesso!')
print(f'Spark versão: {spark.version}')


✅ SparkSession criada com sucesso!
Spark versão: 4.0.0


In [ ]:
# Instalar kagglehub
!pip install -q kagglehub

import kagglehub
import os
import glob

# Download do dataset
print('Baixando dataset do Kaggle...')
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")
print(f'✅ Dataset baixado em: {path}')

# Listar arquivos
files = sorted(glob.glob(path + "/*.csv"))
print(f'\nTotal de arquivos: {len(files)}')
for f in files:
    filename = os.path.basename(f)
    size_mb = os.path.getsize(f) / (1024 * 1024)
    print(f'  - {filename} ({size_mb:.2f} MB)')


Baixando dataset do Kaggle...
Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
✅ Dataset baixado em: /kaggle/input/brazilian-ecommerce

Total de arquivos: 9
  - olist_customers_dataset.csv (8.62 MB)
  - olist_geolocation_dataset.csv (58.44 MB)
  - olist_order_items_dataset.csv (14.72 MB)
  - olist_order_payments_dataset.csv (5.51 MB)
  - olist_order_reviews_dataset.csv (13.78 MB)
  - olist_orders_dataset.csv (16.84 MB)
  - olist_products_dataset.csv (2.27 MB)
  - olist_sellers_dataset.csv (0.17 MB)
  - product_category_name_translation.csv (0.00 MB)


In [ ]:
from pyspark.sql import functions as F

# Carregar datasets necessários
data_files = {}
for f in glob.glob(path + '/*.csv'): # Use a variável 'path' que contém o caminho do Kaggle
    name = os.path.basename(f).replace('.csv', '')
    data_files[name] = spark.read.csv(f, header=True, inferSchema=True, multiLine=True, escape='"')
    print(f'✓ {name}: {data_files[name].count()} linhas')

# Extrair datasets principais
orders = data_files['olist_orders_dataset']
items = data_files['olist_order_items_dataset']
products = data_files['olist_products_dataset']
customers = data_files['olist_customers_dataset']
translations = data_files['product_category_name_translation']

print('\n✅ Datasets carregados com sucesso!')

✓ olist_customers_dataset: 99441 linhas
✓ olist_sellers_dataset: 3095 linhas
✓ olist_order_reviews_dataset: 99224 linhas
✓ olist_order_items_dataset: 112650 linhas
✓ olist_products_dataset: 32951 linhas
✓ olist_geolocation_dataset: 1000163 linhas
✓ product_category_name_translation: 71 linhas
✓ olist_orders_dataset: 99441 linhas
✓ olist_order_payments_dataset: 103886 linhas

✅ Datasets carregados com sucesso!


In [ ]:
# Converter timestamps
orders_ts = (
    orders
    .withColumn("purchase_ts", F.to_timestamp("order_purchase_timestamp"))
    .filter(F.col("purchase_ts").isNotNull())
)

# Extrair features temporais
orders_ts = (
    orders_ts
    .withColumn("year", F.year("purchase_ts"))
    .withColumn("month", F.month("purchase_ts"))
    .withColumn("day_of_week", F.dayofweek("purchase_ts"))
    .withColumn("quarter", F.quarter("purchase_ts"))
    .withColumn("week_of_year", F.weekofyear("purchase_ts"))
)

print("✅ Features temporais criadas")
orders_ts.select("order_id", "year", "month", "quarter", "day_of_week").show(5)


✅ Features temporais criadas
+--------------------+----+-----+-------+-----------+
|            order_id|year|month|quarter|day_of_week|
+--------------------+----+-----+-------+-----------+
|e481f51cbdc54678b...|2017|   10|      4|          2|
|53cdb2fc8bc7dce0b...|2018|    7|      3|          3|
|47770eb9100c2d0c4...|2018|    8|      3|          4|
|949d5b44dbf5de918...|2017|   11|      4|          7|
|ad21c59c0840e6cb8...|2018|    2|      1|          3|
+--------------------+----+-----+-------+-----------+
only showing top 5 rows


In [ ]:
# Juntar orders com items
df_base = (
    orders_ts
    .join(items, on="order_id", how="inner")
    .join(products, on="product_id", how="left")
    .join(customers, on="customer_id", how="left")
    .join(translations, on="product_category_name", how="left")
)

# Selecionar colunas relevantes
df_sales = df_base.select(
    "order_id",
    "year",
    "month",
    "day_of_week",
    "quarter",
    "week_of_year",
    "customer_state",
    F.col("product_category_name_english").alias("category"),
    "price",
    "freight_value",
    "order_item_id"
)

print(f"✅ Base preparada: {df_sales.count()} registros")
df_sales.show(5)


✅ Base preparada: 112650 registros
+--------------------+----+-----+-----------+-------+------------+--------------+---------------+-----+-------------+-------------+
|            order_id|year|month|day_of_week|quarter|week_of_year|customer_state|       category|price|freight_value|order_item_id|
+--------------------+----+-----+-----------+-------+------------+--------------+---------------+-----+-------------+-------------+
|00310b0c75bb13015...|2018|    8|          4|      3|          33|            MG|     housewares| 39.9|        15.38|            1|
|0032d07457ae9c806...|2018|    3|          7|      1|          10|            RJ|     housewares|159.0|        27.19|            1|
|0045e3085f083f0f3...|2017|    8|          5|      3|          32|            SP|  health_beauty|116.9|        13.84|            1|
|0079bca8e89bd52fd...|2018|    5|          5|      2|          18|            SP| sports_leisure| 49.9|         7.39|            1|
|007ff0b0f79be782f...|2018|    2|        

In [ ]:
# Agregar por ano, mês, estado e categoria
sales_agg = (
    df_sales
    .groupBy("year", "month", "customer_state", "category")
    .agg(
        F.sum("price").alias("total_sales"),
        F.count("order_id").alias("num_orders"),
        F.avg("price").alias("avg_price"),
        F.sum("freight_value").alias("total_freight"),
        F.countDistinct("order_id").alias("unique_orders"),
        F.avg("day_of_week").alias("avg_day_of_week")
    )
    .withColumnRenamed("customer_state", "state")
)

print(f"✅ Agregação criada: {sales_agg.count()} registros")
sales_agg.show(5)


✅ Agregação criada: 11808 registros
+----+-----+-----+---------------+------------------+----------+------------------+-----------------+-------------+------------------+
|year|month|state|       category|       total_sales|num_orders|         avg_price|    total_freight|unique_orders|   avg_day_of_week|
+----+-----+-----+---------------+------------------+----------+------------------+-----------------+-------------+------------------+
|2017|   11|   RJ|     cool_stuff|           8169.49|        43|189.98813953488371|923.0599999999998|           43| 4.046511627906977|
|2017|    2|   SP|furniture_decor|           9440.28|       118| 80.00237288135594|          2000.15|           98| 3.805084745762712|
|2017|   10|   PI|   home_confort|              89.9|         1|              89.9|            38.18|            1|               5.0|
|2018|    8|   MG|     stationery|            2056.9|        17|120.99411764705883|           371.45|           17|3.4705882352941178|
|2018|    8|   SP| 

In [ ]:
from pyspark.sql.window import Window

# Criar window particionado por estado e categoria, ordenado por ano e mês
window_spec = Window.partitionBy("state", "category").orderBy("year", "month")

# Criar lag features
sales_agg = (
    sales_agg
    .withColumn("sales_lag_1", F.lag("total_sales", 1).over(window_spec))
    .withColumn("sales_lag_2", F.lag("total_sales", 2).over(window_spec))
    .withColumn("sales_lag_3", F.lag("total_sales", 3).over(window_spec))
    .withColumn("orders_lag_1", F.lag("num_orders", 1).over(window_spec))
    .withColumn("orders_lag_2", F.lag("num_orders", 2).over(window_spec))
)

# Remover linhas com nulls criados pelo lag
sales_agg = sales_agg.na.drop()

print(f"✅ Lag features criadas: {sales_agg.count()} registros")
sales_agg.select("state", "category", "year", "month", "total_sales", "sales_lag_1", "sales_lag_2").show(5)


✅ Lag features criadas: 8085 registros
+-----+--------------------+----+-----+-----------+-----------+-----------+
|state|            category|year|month|total_sales|sales_lag_1|sales_lag_2|
+-----+--------------------+----+-----+-----------+-----------+-----------+
|   AC|computers_accesso...|2017|   10|      51.98|       31.9|      809.1|
|   AC|computers_accesso...|2018|    3|       27.9|      51.98|       31.9|
|   AC|computers_accesso...|2018|    7|      289.0|       27.9|      51.98|
|   AC|         electronics|2018|    2|      13.65|      239.0|      189.9|
|   AC|     furniture_decor|2018|    1|      88.34|      250.0|      179.9|
+-----+--------------------+----+-----+-----------+-----------+-----------+
only showing top 5 rows


In [ ]:
# Média móvel de 3 meses
window_rolling = Window.partitionBy("state", "category").orderBy("year", "month").rowsBetween(-2, 0)

sales_agg = (
    sales_agg
    .withColumn("sales_rolling_mean_3", F.avg("total_sales").over(window_rolling))
    .withColumn("orders_rolling_mean_3", F.avg("num_orders").over(window_rolling))
)

print("✅ Médias móveis criadas")
sales_agg.select("state", "category", "total_sales", "sales_rolling_mean_3").show(5)


✅ Médias móveis criadas
+-----+--------------------+-----------+--------------------+
|state|            category|total_sales|sales_rolling_mean_3|
+-----+--------------------+-----------+--------------------+
|   AC|computers_accesso...|      51.98|               51.98|
|   AC|computers_accesso...|       27.9|               39.94|
|   AC|computers_accesso...|      289.0|              122.96|
|   AC|         electronics|      13.65|               13.65|
|   AC|     furniture_decor|      88.34|               88.34|
+-----+--------------------+-----------+--------------------+
only showing top 5 rows


In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler

# Definir features (removendo 'quarter' que não está em sales_agg)
feature_cols = [
    "year", "month", "avg_day_of_week", # 'quarter' removido daqui
    "num_orders", "avg_price", "total_freight", "unique_orders",
    "sales_lag_1", "sales_lag_2", "sales_lag_3",
    "orders_lag_1", "orders_lag_2",
    "sales_rolling_mean_3", "orders_rolling_mean_3",
    "state_idx", "category_idx"
]

# Indexar variáveis categóricas
idx_state = StringIndexer(inputCol="state", outputCol="state_idx", handleInvalid="keep")
idx_category = StringIndexer(inputCol="category", outputCol="category_idx", handleInvalid="keep")

# Aplicar indexação
sales_indexed = idx_state.fit(sales_agg).transform(sales_agg)
sales_indexed = idx_category.fit(sales_indexed).transform(sales_indexed)

# Assembler
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features_raw",
    handleInvalid="skip"
)

# Scaler
scaler = StandardScaler(inputCol="features_raw", outputCol="features")

# Aplicar transformações
sales_vec = assembler.transform(sales_indexed)
sales_scaled = scaler.fit(sales_vec).transform(sales_vec)

# Dataset final para ML
df_ml = sales_scaled.select("features", F.col("total_sales").alias("label"))

# Split
train, test = df_ml.randomSplit([0.8, 0.2], seed=42)

print(f"✅ Dados preparados para ML")
print(f"   Train: {train.count()} registros")
print(f"   Test: {test.count()} registros")

✅ Dados preparados para ML
   Train: 6540 registros
   Test: 1545 registros


In [ ]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor, DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd

# Definir modelos
models = {
    "Linear Regression": LinearRegression(featuresCol="features", labelCol="label"),
    "Decision Tree": DecisionTreeRegressor(featuresCol="features", labelCol="label", seed=42),
    "Random Forest": RandomForestRegressor(featuresCol="features", labelCol="label", seed=42, numTrees=50),
    "Gradient Boosting": GBTRegressor(featuresCol="features", labelCol="label", seed=42, maxIter=50)
}

# Evaluator
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Treinar e avaliar cada modelo
results_baseline = {}

for name, model in models.items():
    print(f"\nTreinando {name}...")

    # Treinar
    trained_model = model.fit(train)

    # Prever
    predictions = trained_model.transform(test)

    # Avaliar
    r2 = evaluator_r2.evaluate(predictions)
    rmse = evaluator_rmse.evaluate(predictions)

    results_baseline[name] = {"R²": r2, "RMSE": rmse}
    print(f"  R²: {r2:.4f} | RMSE: {rmse:.2f}")

# Mostrar resultados
results_df = pd.DataFrame(results_baseline).T.sort_values("R²", ascending=False)
print("\n📊 Comparação de Modelos (Baseline):")
print(results_df)



Treinando Linear Regression...
  R²: 0.9855 | RMSE: 357.09

Treinando Decision Tree...
  R²: 0.9019 | RMSE: 929.14

Treinando Random Forest...
  R²: 0.9051 | RMSE: 913.96

Treinando Gradient Boosting...
  R²: 0.9322 | RMSE: 772.53

📊 Comparação de Modelos (Baseline):
                         R²        RMSE
Linear Regression  0.985514  357.093578
Gradient Boosting  0.932205  772.526898
Random Forest      0.905109  913.957707
Decision Tree      0.901930  929.140243


In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Escolher Random Forest
rf = RandomForestRegressor(featuresCol="features", labelCol="label", seed=42)

# Grid de hiperparâmetros (reduzido para economizar memória)
paramGrid = (
    ParamGridBuilder()
    .addGrid(rf.numTrees, [50, 100]) # Reduzindo as opções de numTrees
    .addGrid(rf.maxDepth, [5, 10])    # Reduzindo as opções de maxDepth
    .addGrid(rf.minInstancesPerNode, [5, 10]) # Reduzindo e aumentando o valor mínimo
    .build()
)

# CrossValidator (reduzido numFolds para economizar memória)
crossval = (
    CrossValidator(
        estimator=rf,
        estimatorParamMaps=paramGrid,
        evaluator=evaluator_r2,
        numFolds=3, # Reduzindo o número de folds para 3
        seed=42
    )
)

print("🔄 Executando CrossValidation (pode demorar alguns minutos)...")
cv_model = crossval.fit(train)

print("✅ CrossValidation concluída!")

🔄 Executando CrossValidation (pode demorar alguns minutos)...


In [ ]:
# Melhor modelo
best_model = cv_model.bestModel

# Prever no teste
predictions_cv = best_model.transform(test)

# Avaliar
r2_cv = evaluator_r2.evaluate(predictions_cv)
rmse_cv = evaluator_rmse.evaluate(predictions_cv)

print(f"\n🎯 MELHOR MODELO (Random Forest com CrossValidation):")
print(f"   Número de Árvores: {best_model.getNumTrees}")
print(f"   Profundidade Máxima: {best_model.getMaxDepth()}")
print(f"   Min Instances Per Node: {best_model.getMinInstancesPerNode()}")
print(f"   R²: {r2_cv:.4f}")
print(f"   RMSE: {rmse_cv:.2f}")

# Mostrar algumas predições
print("\n📊 Exemplos de Predições:")
predictions_cv.select("label", "prediction").show(10)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Converter para pandas
pred_pd = predictions_cv.select("label", "prediction").toPandas()

# Gráfico de dispersão (Real vs Previsto)
plt.figure(figsize=(10, 6))
plt.scatter(pred_pd["label"], pred_pd["prediction"], alpha=0.5, s=10)
plt.plot([pred_pd["label"].min(), pred_pd["label"].max()],
         [pred_pd["label"].min(), pred_pd["label"].max()],
         'r--', lw=2, label='Linha Ideal')
plt.xlabel("Vendas Reais (R$)")
plt.ylabel("Vendas Previstas (R$)")
plt.title(f"Previsão vs Real (R² = {r2_cv:.4f})")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Gráfico de resíduos
pred_pd['residual'] = pred_pd['label'] - pred_pd['prediction']

plt.figure(figsize=(10, 6))
plt.scatter(pred_pd["prediction"], pred_pd["residual"], alpha=0.5, s=10)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel("Vendas Previstas (R$)")
plt.ylabel("Resíduos (Real - Previsto)")
plt.title("Análise de Resíduos")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Adicionar resultado do CV
results_baseline["Random Forest (CV)"] = {"R²": r2_cv, "RMSE": rmse_cv}

# Criar DataFrame
results_final = pd.DataFrame(results_baseline).T.sort_values("R²", ascending=False)

print("\n📊 Resultados Finais - Comparação de Modelos:")
print(results_final)

# Gráfico de barras
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# R²
axes[0].barh(results_final.index, results_final["R²"], color='steelblue')
axes[0].axvline(x=0.65, color='red', linestyle='--', label='Meta (0.65)')
axes[0].set_xlabel("R²")
axes[0].set_title("Comparação de Modelos - R²")
axes[0].legend()
axes[0].grid(axis='x', alpha=0.3)

# RMSE
axes[1].barh(results_final.index, results_final["RMSE"], color='coral')
axes[1].set_xlabel("RMSE")
axes[1].set_title("Comparação de Modelos - RMSE")
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Feature importances
importances = best_model.featureImportances.toArray()

feature_importance_df = pd.DataFrame({
    "feature": feature_cols,
    "importance": importances
}).sort_values("importance", ascending=False)

# Top 15 features
top_features = feature_importance_df.head(15)

plt.figure(figsize=(10, 8))
plt.barh(top_features["feature"][::-1], top_features["importance"][::-1], color='teal')
plt.xlabel("Importância")
plt.title("Top 15 Features Mais Importantes - Random Forest")
plt.tight_layout()
plt.show()

print("\n📊 Top 15 Features mais importantes:")
print(top_features.to_string(index=False))


In [ ]:
# Salvar modelo
model_path = "/content/best_sales_rf_model"
best_model.write().overwrite().save(model_path)

# Compactar
import shutil
shutil.make_archive("best_sales_rf_model", 'zip', model_path)

print("✅ Modelo salvo e compactado!")
print(f"   Arquivo: best_sales_rf_model.zip")


In [ ]:
!pip install -q ipywidgets

import ipywidgets as widgets
from IPython.display import display

# Obter listas únicas
states_list = [row.state for row in sales_agg.select('state').distinct().collect()]
categories_list = [row.category for row in sales_agg.select('category').distinct().collect()]

# Widgets
state_widget = widgets.Dropdown(options=sorted(states_list), description='Estado:')
category_widget = widgets.Dropdown(options=sorted(categories_list), description='Categoria:')
year_widget = widgets.IntSlider(value=2018, min=2016, max=2020, description='Ano:')
month_widget = widgets.IntSlider(value=6, min=1, max=12, description='Mês:')
predict_button = widgets.Button(description='Fazer Previsão', button_style='success')
output_widget = widgets.Output()

def on_predict_button_clicked(b):
    with output_widget:
        output_widget.clear_output()

        state = state_widget.value
        category = category_widget.value
        year = year_widget.value
        month = month_widget.value

        # Buscar dados históricos para calcular lags
        hist = sales_agg.filter(
            (F.col("state") == state) &
            (F.col("category") == category)
        ).orderBy("year", "month").tail(3)

        if len(hist) < 3:
            print("⚠️ Dados históricos insuficientes para esta combinação")
            return

        # Criar features baseadas nos últimos 3 meses
        input_data = {
            'year': year,
            'month': month,
            'quarter': (month - 1) // 3 + 1,
            'avg_day_of_week': hist[-1]['avg_day_of_week'],
            'num_orders': hist[-1]['num_orders'],
            'avg_price': hist[-1]['avg_price'],
            'total_freight': hist[-1]['total_freight'],
            'unique_orders': hist[-1]['unique_orders'],
            'sales_lag_1': hist[-1]['total_sales'],
            'sales_lag_2': hist[-2]['total_sales'] if len(hist) > 1 else 0,
            'sales_lag_3': hist[-3]['total_sales'] if len(hist) > 2 else 0,
            'orders_lag_1': hist[-1]['num_orders'],
            'orders_lag_2': hist[-2]['num_orders'] if len(hist) > 1 else 0,
            'sales_rolling_mean_3': hist[-1]['sales_rolling_mean_3'],
            'orders_rolling_mean_3': hist[-1]['orders_rolling_mean_3'],
            'state': state,
            'category': category
        }

        # Criar DataFrame Spark
        input_df = spark.createDataFrame([input_data])

        # Aplicar transformações
        input_indexed = idx_state.fit(input_df).transform(input_df)
        input_indexed = idx_category.fit(input_indexed).transform(input_indexed)
        input_vec = assembler.transform(input_indexed)
        input_scaled = scaler.fit(input_vec).transform(input_vec)

        # Fazer previsão
        prediction_result = best_model.transform(input_scaled)
        predicted_sales = prediction_result.select('prediction').first()[0]

        print(f"📊 PREVISÃO DE VENDAS")
        print(f"   Estado: {state}")
        print(f"   Categoria: {category}")
        print(f"   Período: {month}/{year}")
        print(f"   Vendas Previstas: R$ {predicted_sales:,.2f}")

        # Mostrar histórico
        print(f"\n📈 Histórico recente:")
        for i, h in enumerate(hist[-3:], 1):
            print(f"   {h['month']}/{h['year']}: R$ {h['total_sales']:,.2f}")

predict_button.on_click(on_predict_button_clicked)

# Exibir interface
ui = widgets.VBox([
    widgets.HBox([state_widget, category_widget]),
    widgets.HBox([year_widget, month_widget]),
    predict_button,
    output_widget
])

print('🎯 Use os controles abaixo para fazer previsões:')
display(ui)
